In [6]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(dotenv_path=find_dotenv(".env.local"))


True

## 调用OpenAI API（访问DeepSeek）

In [7]:
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY"), 
    base_url=os.getenv("DEEPSEEK_API_URL")
)

completion = client.chat.completions.create(
    model=os.getenv("DEEPSEEK_MODEL"),
    messages=[
        {"role": "system", "content": "You are a helpful assistnat."},
        {"role": "user", "content": "Hello!"}
    ]
)

completion


ChatCompletion(id='bb718432-5896-4759-b390-9f3ff07657f0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! 😊 How can I assist you today?', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1747749798, model='deepseek-chat', object='chat.completion', service_tier=None, system_fingerprint='fp_8802369eaa_prod0425fp8', usage=CompletionUsage(completion_tokens=11, prompt_tokens=12, total_tokens=23, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0), prompt_cache_hit_tokens=0, prompt_cache_miss_tokens=12))

In [8]:
print(completion.choices[0].message.content)

Hello! 😊 How can I assist you today?


In [9]:
# 做一些简单的封装
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY"), 
    base_url=os.getenv("DEEPSEEK_API_URL")
)

def get_gpt_messages(prompt):
    '''
    构造 GPT 模型请求参数 messages

    请求参数：
        prompt: 对应的用户提示词
    '''
    messages = [{"role": "user", "content": prompt}]
    return messages

def get_completion(prompt, model="deepseek-chat", temperature=0.):
    '''
    获取 GPT 模型调用结果

    请示参数：
        prompt: 对应的提示词
        model: 调用的模型，默认为 deepseek-chat, 也可以按需选择其它模型
        temperature: 模型输出的温度系数, 控制输出的随机程度, 取值范围为 [0, 2.0], 温度系数越低, 输出内容越一致。
    '''
    response = client.chat.completions.create(
        model=model,
        messages=get_gpt_messages(prompt),
        temperature=temperature,
    )
    if len(response.choices) > 0:
        return response.choices[0].message.content
    return "generate answer error"


In [10]:
get_completion("你好")

'你好！😊 很高兴见到你～有什么我可以帮你的吗？'